CÓDIGO PYTHON WEMOB.

El objetivo del siguiente código es recoger la depuración realizada en notebook anterior (que da como resultado un archivo .csv denominado wemob) que contiene todos los timestamp facilitados por WEMOB que pasaron dicho filtro (Notebook : TFMwemob1), y que ahora comenzaremos a sintetizar para conseguir unos datasets que permitan los procesos posteriores : EVALUACIÓN RUTAS MÁS REPETIDAS, ALIMENTACIÓN ALGORITMO PREDICTIVO DEL CONSUMO DE LAS MISMAS.

In [ ]:
# Cargamos las diferentes librerias que vamos a necesitar.

import pandas as pd
import numpy as np
import json
import glob
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from datetime import timedelta,datetime
import seaborn as sns
import datetime
import gc

# Para permitir ver todas las columnas/filas posibles, que permitan diseccionar los diferentes datasets resultantes
# a la busqueda de posibles errores o malas interpretaciones.

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
# Cargamos paquete que se nos solicita en Cloud.

pip install gcsfs

In [ ]:
# Primer contador.

print(datetime.datetime.now())

Secuencia tareas a realizar:

    A) Creación dataset (DESARROLLO RUTAS), que aglutinará el detalle del desarrollo de las mismas en función de su      
    agrupamiento (dependiendo de la situación).
    
    Situaciones: 1 (Paro total, no hay velocidad no hay revoluciones por minuto).
                 2 (Paro a ralentí, no hay velocidad pero si revoluciones por minuto para mantener refrigerada la caja).
                 3 (En ruta, hay velocidad y por tanto entendemos que el camión está en movimiento).
                 
    B) Una vez tenemos reducidos los registros del dataset original, en función de su situación.....identificamos las rutas.

    El parámetro más importante es la duración de la parada (ya sea total o al ralentí), en este caso vamos a establecer           mayor o igual a una hora, pero hemos de tener en cuenta que si ampliamos dicha duración las rutas se reducirán                 ostensiblemente (dependiente de la duración que se estime).

    Ejemplo, para una ruta su timestamp de incio será el TMPc del estado y su timestamp final será el TMPa del estado.

    Nos aseguramos que las columnas TMPa y TMPc tienen valor datetime, y salvo las variables catalogadas como float el resto       se convierten en integer, manteniendo el mismo criterio que en la depuración para optimizar al máximo el rendimiento           computacional.

    Añadimos al dataframe una columna que será la diferencia de fechas, evaluada en horas.El resultado sería una nueva             columna tipo timedelta, y la convertimos a float.
    
    C) Este código, lo que hace es relacionar cada ruta definida en RUTAS con su secuencia de desarrollo en M2                     (DESARROLLO_RUTAS), estableciendo como campo de enlace NRUTA (idéntico en ambos dataset) con la intencion de poder             establecer consultas a futuro.
    
    D) Se carga la librería GEOPY con objeto de establecer (en función de la latitud/longitud), el código postal/comunidad/pais 
    de todas y cada una de las lineas contenidas en el dataset RUTAS. Esta parte del código tiene un consumo computaciones de       unas 8 hrs, debido a las rutas identificadas.
    
    E) Agrupando el dataset RUTAS por variables de salida y llegada, se establece un ranking (rutas más habituales) que se         almacenará en el dataset MRUTAS.
    
    F) El objetivo será crear un dataset (MAESTRO_RUTAS general) que sintetice desarrollo de rutas (M2) en una sola ruta, por       matricula y donde las variables resultantes sean promedios o diferencias en función de la naturaleza de las mismas. Este       dataset simplifica y concentra la información con objeto de poderse modelizar de una forma más eficiente.
    
    G) En este caso, el dataset resultante (MAESTRO_RUTAS x conductor) tiene la misma finalidad pero incluyendo los datos           particulares de los diferentes conductores que hayan sido asignados a las rutas.

In [ ]:
# Leemos csv reseñado al inicio, producto de depuración procesada por notebook TFMwemob1.

M2 = pd.read_csv('wemob.csv')


In [ ]:
# Borramos la columna de exceso.

del(M2['Unnamed: 0'])


In [ ]:
M2.sort_values(['MatriculaNum','Timestamp'], inplace=True)

In [ ]:
M2 = M2.reset_index()
del M2['index']

In [ ]:
# Nos aseguramos que Timestamp tiene formato datetime
M2['Timestamp'] = pd.to_datetime(M2['Timestamp'])

In [ ]:
M2.dtypes

In [ ]:
M2.shape

# TAREA (A)

El objetivo es reducir el número de registros del dataset inicial, en función de la situación (parada total - cuando velocidad es cero y las rpm también - / parada ralentí - cuando velocidad es cero y las rpm no lo son - / parada total - cuando el transporte está en marcha ) y los diferentes timestamp consecutivos que cumplen las mismas condiciones.

In [ ]:
M2['CS']=0

# Renombramos columnas del dataset resultante.
M2.rename(columns={'Latitud':'LATa', 'Longitud':'LONGa', 'Altitud':'ALTa', 'Odometro':'ODOa', 'Velocidad':'VELCa',\
'HorasMotor':'HMa','ControlCrucero':'CCa', 'TemperaturaMotor':'TMa', 'RPM':'RPMa', 'NivelFuel':'NFa','ConsumoTotal':'CTa',\
'Frenadas':'FRNa', 'TiempoRalenti':'TRa', 'ParMotor':'PMa', 'Pedal':'PDLa','Embrague':'EMBa', 'ConductorNum':'COND',\
'MatriculaNum':'MTR', 'Timestamp':'TMPa'},inplace=True)


In [ ]:
# Creamos las variables faltantes.

M2['SEC']= M2.index
M2['NRUTA']=0
M2['POSICION']=0
M2['LATc']= M2['LATa']
M2['LONGc']= M2['LONGa']
M2['ALTc']= M2['ALTa']
M2['ODOc']= M2['ODOa']
M2['VELCc']= M2['VELCa']
M2['HMc']= M2['HMa']
M2['CCc']= M2['CCa']
M2['TMc']= M2['TMa']
M2['RPMc']= M2['RPMa']
M2['NFc']= M2['NFa']
M2['CTc']= M2['CTa']
M2['FRNc']= M2['FRNa']
M2['TRc']= M2['TRa']
M2['PMc']= M2['PMa']
M2['PDLc']= M2['PDLa']
M2['EMBc']= M2['EMBa']
M2['TMPc']= M2['TMPa']



In [ ]:
# Reordenamos todas las columnas.
M2 = M2[['SEC','NRUTA','POSICION','MTR','CS','LATa','LATc','LONGa','LONGc','TMPa','TMPc','COND','ALTa',\
                           'ALTc','ODOa','ODOc','VELCa','VELCc','HMa','HMc','CCa','CCc','TMa','TMc','RPMa','RPMc','NFa',\
                           'NFc','CTa','CTc','FRNa','FRNc','TRa','TRc','PMa','PMc','PDLa','PDLc','EMBa','EMBc']]

In [ ]:
# Establecemos registros tipo 1-2-3

M2.loc[M2.RPMa == 0, ['CS']] = 1
M2.loc[M2.RPMa != 0, ['CS']] = 2
M2.loc[M2.VELCa != 0, ['CS']] = 3

In [ ]:
# Establecemos indice para aglutinar registros. En este caso utilizaremos latitud y longitud,
# como es lógico son diferentes en cada estado.
GBcs0 = M2.groupby( [ 'LATa', 'LONGa','ODOa','CS','MTR','COND'] )

# Variables iniciales.
cols = ['SEC']

for c in cols:
    M2[c] = GBcs0[c].transform("first")

In [ ]:
# Aglutinamos entonces.

GBcs0 = M2.groupby( ['SEC'] )

# Variables iniciales.
cols = ['MTR','CS','LATa','LONGa','TMPa','COND','ALTa','ODOa', 'VELCa', 'HMa','CCa', 'TMa', 'RPMa','NFa','CTa', 'FRNa','TRa', 'PMa','PDLa', 'EMBa']
for c in cols:
    M2[c] = GBcs0[c].transform("first")

# Variables finales
cols = ['LATc','LONGc','TMPc','ALTc','ODOc', 'VELCc', 'HMc','CCc', 'TMc', 'RPMc','NFc','CTc','FRNc','TRc', 'PMc','PDLc', 'EMBc']
for c in cols:
    M2[c] = GBcs0[c].transform("last")

# Realizamos la primera depuración.

M2= M2.drop_duplicates()


In [ ]:
M2.shape

In [ ]:
M2.head()

In [ ]:
df = M2[['TMPc']]
df.rename(columns={'TMPc': 'TMPx'}, inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Eliminamos la primera linea para que luego cuando se proceda con el merge (uniendo M2/df) podamos emparejar las variables
# correctar para calcular TMPx

M2 = M2.drop([0],axis=0)

In [ ]:
M2 = M2.reset_index()
del M2['index']

In [ ]:
df = df.reset_index()
del df['index']

In [ ]:
M2.to_csv('M2.csv')
df.to_csv('df.csv')
print ('Proceso concluido')
gc.collect()
print(datetime.datetime.now())

# Tarea (B)

In [ ]:
M2 = pd.merge(M2, df, right_index=True, left_index=True)

In [ ]:
M2.head()

In [ ]:
# Ordenamos el dataset RUTAS y DESARROLLO_RUTAS (M2), para asegurar que el código asigna rutas correctamente
M2.sort_values(['MTR','TMPc'], inplace=True)

In [ ]:
M2['days']= M2['TMPc']-M2['TMPa']
M2['S']= M2['TMPa']-M2['TMPx']

# Creamos una variable que coniene el tiempo de desarrollo de esa linea (se utilizará para la identificación de rutas).

M2['TIME_HRS'] = M2['days'] / np.timedelta64(1, 'h')

# Creamos una variable que contiene el tiempo cronologico entre linea y linea (se utilizará para la identificación de rutas).

M2['DELAYED'] = M2['S']/np.timedelta64(1, 'h')

del(M2['days'])
del(M2['SEC'])
del(M2['S'])
del(M2['TMPx'])

Iniciamos la identificación de rutas. Pero antes vamos a ordenar por matricua y primer timestamp.

In [ ]:
# Creamos el dataset Rutas

RUTAS = pd.DataFrame(columns=('MTR','LATorigen','LONGorigen','LATdestino','LONGdestino','FECHA_HORAsalida','FECHA_HORAllegada',\
                              'DIRECCsalida','DIRECCllegada','LATLONGs','LATLONGll','NRUTA','DCRUTA'))

In [ ]:
DC = 0
A = 0
Iord = 0 # Esta variable controlará la elección de filas del dataframe.
Tr = 1

In [ ]:
####################################### COMIENZO ANALISIS
# PUESTA EN MARCHA
# Nos ponemos en marcha. Le damos un valor a las variables de control iniciales.

if M2.iloc[Iord,3] <3:
    
    MATa = M2.iloc[Iord,2]
    TMPa = M2.iloc[Iord,9]
    LATa = M2.iloc[Iord,5]
    LONGa = M2.iloc[Iord,7]
else:
    MATa = M2.iloc[Iord,2]
    TMPa = M2.iloc[Iord,8]
    LATa = M2.iloc[Iord,4]
    LONGa = M2.iloc[Iord,6]
    
    
# COMIENZA EL CODIGO

while Tr <= len(M2.index):

# CONTROL MATRICULA

# Si es el primer registro saltamos todo el código
    if A == 0:
        A = A + 1
        DC = 1

# Si las matriculas no coinciden, estaremos ante un registro impar y deberemos resetear el sistema.

    if (M2.iloc[Iord,2] != MATa) and DC == 0:
        
        # Retrocedemos una linea.
        
        Iord = Iord - 1
        
        # ANTES DEBEMOS GRABAR RUTA FINAL
        RUTAS=RUTAS.append({'MTR' : MATa , 'FECHA_HORAsalida' : TMPa , 'LATorigen' : LATa ,'LONGorigen' : LONGa ,\
        'FECHA_HORAllegada' : M2.iloc[Iord,9] , 'LATdestino' : M2.iloc[Iord,5],'LONGdestino' : M2.iloc[Iord,7],\
        'DIRECCsalida' : '','DIRECCllegada' : '','LATLONGs' : '','LATLONGll' : '','NRUTA' : 0,'DCRUTA' : 0 } ,\
        ignore_index=True) 
       
        # RESETEAMOS VARIABLES
        
        # Nos ubicamos en la línea siguiente.
        
        Iord = Iord + 1
        
        if M2.iloc[Iord,3] <3:
    
            MATa = M2.iloc[Iord,2]
            TMPa = M2.iloc[Iord,9]
            LATa = M2.iloc[Iord,5]
            LONGa = M2.iloc[Iord,7]
        else:
            MATa = M2.iloc[Iord,2]
            TMPa = M2.iloc[Iord,8]
            LATa = M2.iloc[Iord,4]
            LONGa = M2.iloc[Iord,6]
            
    # EN CASO CONTRARIO VALORAMOS LAS POSIBLES SITUACIONES QUE DEBEMOS CONTEMPLAR.
  
    if (M2.iloc[Iord,2] == MATa) and (M2.iloc[Iord,3] <3) and (M2.iloc[Iord,39] >= 1) and DC == 0:
        # GRABAMOS REGISTRO RESUMEN
        RUTAS=RUTAS.append({'MTR' : MATa , 'FECHA_HORAsalida' : TMPa , 'LATorigen' : LATa ,'LONGorigen' : LONGa ,\
        'FECHA_HORAllegada' : M2.iloc[Iord,9] , 'LATdestino' : M2.iloc[Iord,5],'LONGdestino' : M2.iloc[Iord,7],\
        'DIRECCsalida' : '','DIRECCllegada' : '','LATLONGs' : '','LATLONGll' : '','NRUTA' : 0,'DCRUTA' : 0 } ,\
        ignore_index=True) 
        # REGULARIZAMOS NUEVAS VARIABLES COMO ORIGEN PROXIMA RUTA SI COINCIDE LA MATRICULA
        MATa = M2.iloc[Iord,2]
        TMPa = M2.iloc[Iord,9]
        LATa = M2.iloc[Iord,5]
        LONGa = M2.iloc[Iord,7]
    
    if  M2.iloc[Iord,40] >= 1 and DC == 0:
        
        # Retrocedemos una linea.
        
        Iord = Iord - 1
        
        # ANTES DEBEMOS GRABAR RUTA FINAL
        RUTAS=RUTAS.append({'MTR' : MATa , 'FECHA_HORAsalida' : TMPa , 'LATorigen' : LATa ,'LONGorigen' : LONGa ,\
        'FECHA_HORAllegada' : M2.iloc[Iord,9] , 'LATdestino' : M2.iloc[Iord,5],'LONGdestino' : M2.iloc[Iord,7],\
        'DIRECCsalida' : '','DIRECCllegada' : '','LATLONGs' : '','LATLONGll' : '','NRUTA' : 0,'DCRUTA' : 0 } ,\
        ignore_index=True) 
       
        # RESETEAMOS VARIABLES
        # Nos ubicamos en la línea siguiente.
        Iord = Iord + 1
        # REGULARIZAMOS NUEVAS VARIABLES COMO ORIGEN PROXIMA RUTA SI COINCIDE LA MATRICULA
        MATa = M2.iloc[Iord,2]
        TMPa = M2.iloc[Iord,9]
        LATa = M2.iloc[Iord,5]
        LONGa = M2.iloc[Iord,7]
        
    
    # Avanzamos registro.
    DC = 0
    Iord = Iord + 1
    Tr = Tr + 1

In [ ]:
print (Iord)

In [ ]:
RUTAS.head(10)

In [ ]:
RUTAS.shape

In [ ]:
RUTAS.to_csv( 'RUTAST0g.csv' )        
M2.to_csv('DESARROLLO_RUTAST0.csv')
gc.collect()
print ('Proceso concluido')
print(datetime.datetime.now())

# Tarea (C)

Comenzamos el nuevo código.

In [ ]:
# VARIABLES CONTROL Y DESARROLLO

# CONTADOR RUTA
CR = 0

# CONTADOR POSICION DENTRO DE LA RUTA
CPS = 0

# VARIOS 

DC = 0
A = 0
Iord = 0 # Esta variable controlará la elección de filas del dataframe M2.
Iordd = 0 # Esta variable controlará la elección de filas del dataframe RUTAS
Tr = 1

# COMIENZO ANALISIS

CR = CR + 1

while Tr <= len(RUTAS.index):
    
    while A < 1:
        
        if (RUTAS.iloc[Iordd,0] == M2.iloc[Iord,2])  and \
        (M2.iloc[Iord,9] <= RUTAS.iloc[Iordd,6]):
            
            M2.at[Iord,'NRUTA'] = CR
            M2.at[Iord,'POSICION'] = CPS    
            
        if len(M2.index) == Iord or (RUTAS.iloc[Iordd,0] != M2.iloc[Iord,2]) or (M2.iloc[Iord,9] > RUTAS.iloc[Iordd,6]) :
            
            A = 1
         
        else:
            
            CPS = CPS + 1
            Iord = Iord + 1 
        
    
    # Modificamos registro de RUTAS y avanzamos.
    
    RUTAS.at[Iordd,'NRUTA'] = CR
    A = 0
    CR = CR + 1
    CPS = 0
    Iordd = Iordd + 1   
    Tr = Tr + 1



In [ ]:
M2.to_csv('DESARROLLO_RUTAST1.csv') 
RUTAS.to_csv ('RUTAST1g.csv')
gc.collect()
print ('Proceso concluido')  
print(datetime.datetime.now())

# Tarea (D)

In [ ]:
# Cumplimentamos las variables que tendremos que utilizar para GEOPY

RUTAS['LATLONGs'] = RUTAS ['LATorigen'].astype(str)+','+ RUTAS ['LONGorigen'].astype(str)
RUTAS['LATLONGll'] = RUTAS ['LATdestino'].astype(str)+','+ RUTAS ['LONGdestino'].astype(str)

In [ ]:
pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = 'TFM')


In [ ]:
RUTAS['DIRECCsalida'] = RUTAS['LATLONGs'].apply(lambda x : geolocator.reverse(x).raw ['address'])
RUTAS['DIRECCllegada'] = RUTAS['LATLONGll'].apply(lambda x : geolocator.reverse(x).raw ['address'])

In [ ]:
RUTAS.head()

In [ ]:
# Preparamos la información que debemos trasladar a DIRECCsalida, conteniendo el código de provincia, la comunidad y el pais.
a = len(RUTAS.index)
x=1
Iord = 0
while x <= a:
    try:
        b = RUTAS['DIRECCsalida'][Iord]['country']
    except KeyError: 
        b=""
    try:
        b1 = RUTAS['DIRECCsalida'][Iord]['state']
    except KeyError: 
        b1=""
    try:
        b2 = RUTAS['DIRECCsalida'][Iord]['postcode']
    except KeyError: 
        b2=""    
    RUTAS.at[Iord,'DIRECCsalida']=b2+'/'+b+','+b1
    Iord = Iord + 1
    x = x +1


In [ ]:
# Preparamos la información que debemos trasladar a DIRECCllegada, conteniendo el código de provincia, la comunidad y el pais.
a = len(RUTAS.index)
x=1
Iord = 0
while x <= a:
    try:
        b = RUTAS['DIRECCllegada'][Iord]['country']
    except KeyError: 
        b=""
    try:
        b1 = RUTAS['DIRECCllegada'][Iord]['state']
    except KeyError: 
        b1=""
    try:
        b2 = RUTAS['DIRECCllegada'][Iord]['postcode']
    except KeyError: 
        b2=""    
    RUTAS.at[Iord,'DIRECCllegada']=b2+'/'+b+','+b1
    Iord = Iord + 1
    x = x +1


In [ ]:
RUTAS.to_csv ('RUTAST2gbc.csv')
gc.collect()
print ('Proceso concluido')  
print(datetime.datetime.now())

# Tarea (E)

Lo primero que debemos hacer es crear un nuevo dataframe para proceder a las agrupaciones de las rutas, y poder confeccionar el ranking de las rutas más habituales, indicando el DCRUTA.

In [ ]:
# Leemos RUTAS para evitar consumo computacional.
M2 = pd.read_csv('DESARROLLO_RUTAST1.csv')
RUTAS = pd.read_csv('RUTAST2gbc.csv')
del(M2['Unnamed: 0'])
del(RUTAS['Unnamed: 0'])

In [ ]:
RUTAS['FECHA_HORAsalida'] = pd.to_datetime(RUTAS['FECHA_HORAsalida'])
RUTAS['FECHA_HORAllegada'] = pd.to_datetime(RUTAS['FECHA_HORAllegada'])


In [ ]:
# CREAMOS EL DATAFRAME A TRAVÉS DE UN FILTRO DE RUTAS

RUTASagr =  RUTAS.filter(items=['DIRECCsalida','DIRECCllegada'])  


In [ ]:
RUTASagr['DIRECCsalida'] = RUTASagr['DIRECCsalida'] .astype(str)
RUTASagr['DIRECCllegada'] = RUTASagr['DIRECCllegada'] .astype(str)

In [ ]:
RUTAS.dtypes

In [ ]:
RUTASagr.head(50)

In [ ]:
RUTASagr.shape

In [ ]:
RUTASagr = RUTASagr.groupby( [ 'DIRECCsalida','DIRECCllegada'] )

MRUTAS =pd.DataFrame(RUTASagr.size().reset_index(name = 'Count'))

# El campo Count contabilizará las veces que se repite dicha ruta, y posteriormente se ordena de forma descendente por el
# campo Count.

MRUTAS.sort_values(['Count'],ascending = False, inplace = True)

# Como el index está un pelín descolocado, lo reseteamos.

MRUTAS = MRUTAS.reset_index()
del MRUTAS['index']

# Creamos otro campo, donde normalizar el ranking.

MRUTAS['DCRUTAS']=0
  

In [ ]:
MRUTAS.dtypes

In [ ]:
MRUTAS.head()

In [ ]:

Iord = 0 # Esta variable controlará la elección de filas del dataframe RUTAS.
Tr = 1 # Esta variable controlará el bucle de filas del dataframe RUTAS.
F = 1 # Esta variable controlará el valor de DCRUTA
Fa = 1 # Esta variable controlará la evolución de Count
A=0
# INICIO CODIGO

while Tr <= len(MRUTAS.index):
    if A == 0:
        A=1
        MRUTAS.iloc[Iord,3] = Tr 
        Fa = MRUTAS.iloc[Iord,2]
    else:
        if Fa == MRUTAS.iloc[Iord,2]:
            MRUTAS.iloc[Iord,3] = F 
            Fa = MRUTAS.iloc[Iord,2]
        else:
            F=F+1
            MRUTAS.iloc[Iord,3] = F
            Fa = MRUTAS.iloc[Iord,2]
        
    Iord = Iord+1
    Tr = Tr+1

gc.collect()
print ('proceso concluido') 
print(datetime.datetime.now())


In [ ]:
# MARCAMOS RUTAS SEGÚN SU RANKING

Iord = 0 # Esta variable controlará la elección de filas del dataframe RUTAS.
Tr = 1 # Esta variable controlará el bucle de filas del dataframe RUTAS.

Iordd = 0 # Esta variable controlará la elección de filas del dataframe MRUTAS.
Trd = 1 # Esta variable controlará el bucle de filas del dataframe MRUTAS.

B=0

# INICIO CODIGO

while Tr <= len(RUTAS.index):
    Iordd = 0
    Trd = 1
    while Trd <= len(MRUTAS.index):
        if ((MRUTAS.iloc[Iordd,0] == RUTAS.iloc[Iord,7])  and\
            (MRUTAS.iloc[Iordd,1] == RUTAS.iloc[Iord,8])): 
            
            RUTAS.iloc[Iord,12] = MRUTAS.iloc[Iordd,3]
            
            
        Iordd = Iordd + 1 
        Trd = Trd + 1
     
    
    Iord = Iord + 1
    Tr = Tr + 1




In [ ]:
RUTAS.to_csv ('RUTAST3g.csv')  
MRUTAS.to_csv ('MRUTAST0g.csv')
gc.collect()
print ('proceso concluido')
print(datetime.datetime.now())

# Tarea (F)

In [ ]:
# Lo primero creamos un dataframe donde el NRUTA sera distinto a 0, para trabajar sobre él.
M2bis = M2
M2bis = M2bis.drop(M2bis[M2bis['POSICION']==0].index)

In [ ]:
# DEFINICIÓN VARIABLE COMPOSICIÓN DATASET
# Para todos el nexo de unión será la ruta.

# CONDUCTORES
# Cuantos conductores han estado asignados a cada ruta.(GBcond).

GBcond= M2bis.groupby(by='NRUTA', as_index=False).agg({'COND': pd.Series.nunique})
GBcond= GBcond.drop_duplicates()

# Para todos el nexo de unión será la ruta

# Tipo de situaciones en cada ruta, y tiempo acumulado de las mismas. implicados en cada ruta GBmat
GBcs = M2bis[['NRUTA','CS','TIME_HRS']] 
GBcs0 = GBcs.groupby( [ 'NRUTA','CS'] )
GBcs['CShrssum'] =GBcs0['TIME_HRS'].transform("sum")
GBcs['CShrsmean'] =GBcs0['TIME_HRS'].transform("mean")
GBcs['CSrept'] =GBcs0['CS'].transform("count")

# Generamos dataset con CS tipo 1
GBcs1 = GBcs[(GBcs['CS'] == 1)]
GBcs1 = GBcs1[['NRUTA','CS','CShrssum','CShrsmean','CSrept']]
GBcs1.rename(columns={'CS': 'CS1','CShrssum':'CS1hrssum','CShrsmean':'CS1hrsmean','CSrept':'CS1rept'}, inplace=True)
GBcs1= GBcs1.drop_duplicates()

# Generamos dataset con CS tipo 2
GBcs2 = GBcs[(GBcs['CS'] == 2)]
GBcs2 = GBcs2[['NRUTA','CS','CShrssum','CShrsmean','CSrept']]
GBcs2.rename(columns={'CS': 'CS2','CShrssum':'CS2hrssum','CShrsmean':'CS2hrsmean','CSrept':'CS2rept'}, inplace=True)         
GBcs2= GBcs2.drop_duplicates()

# Generamos dataset con CS tipo 3             
GBcs3 = GBcs[(GBcs['CS'] == 3)]
GBcs3 = GBcs3[['NRUTA','CS','CShrssum','CShrsmean','CSrept']]
GBcs3.rename(columns={'CS': 'CS3','CShrssum':'CS3hrssum','CShrsmean':'CS3hrsmean','CSrept':'CS3rept'}, inplace=True)         
GBcs3= GBcs3.drop_duplicates()

# Acumulado variables restantes. Se realizara un promedio de cada una de ella.

GBrv = M2bis[['NRUTA','ALTa', 'ALTc', 'VELCa', 'HMa', 'HMc','CCa', 'CCc', 'TMa', 'RPMa','NFa', 'FRNa', 'FRNc',\
        'TRa', 'TRc', 'PMa', 'PMc','PDLa','PDLc', 'EMBa', 'EMBc']]

GBrv0 = GBrv.groupby( [ 'NRUTA' ] )

# Asignamos promedios

cols = ['VELCa', 'TMa', 'RPMa','NFa','TRa']

for c in cols:
    GBrv[c] = GBrv0[c].transform("mean")
    
# Asignamos valores iniciales.

cols = ['ALTa', 'HMa','FRNa','PMa', 'PDLa', 'EMBa']

for c in cols:
    GBrv[c] = GBrv0[c].transform("first")

GBrv['CCa'] = GBrv0['CCa'].transform("sum")

# Asignamos valores finales.

cols = ['ALTc', 'HMc','FRNc','PMc', 'PDLc', 'EMBc']

for c in cols:
    GBrv[c] = GBrv0[c].transform("last")

GBrv['CCc'] = GBrv0['CCc'].transform("count") 

GBrv= GBrv.drop_duplicates()

GBrv['ALTf'] = GBrv['ALTc']-GBrv['ALTa']
GBrv['HMf'] = GBrv['HMc']-GBrv['HMa']
GBrv['CCf'] = (GBrv['CCa']/GBrv['CCc'])
GBrv['FRNf'] = GBrv['FRNc']-GBrv['FRNa'] # Este valor se entendería como porcentaje.
GBrv['PMf'] = GBrv['PMc']-GBrv['PMa']
GBrv['PDLf'] = GBrv['PDLc']-GBrv['PDLa']
GBrv['EMBf'] = GBrv['EMBc']-GBrv['EMBa']

GBrv = GBrv[['NRUTA','ALTf', 'VELCa', 'HMf','CCf', 'TMa', 'RPMa','NFa', 'FRNf',\
        'TRa', 'PMf','PDLf', 'EMBf']]

GBrv= GBrv.drop_duplicates()

# Manipulamos las variables que mantuvimos como float. ODO (odómetro) y CT (consumo total), estableciendo valor inicial y valor final.

GBvp = M2bis[['NRUTA','ODOa', 'ODOc', 'CTa','CTc']]
GBvp0 = GBvp.groupby(['NRUTA'])
GBvp['ODOa'] =GBvp0['ODOa'].transform("first")
GBvp['ODOc'] =GBvp0['ODOc'].transform("last")
GBvp['CTa'] =GBvp0['CTa'].transform("first")
GBvp['CTc'] =GBvp0['CTc'].transform("last")
GBvp= GBvp.drop_duplicates()
GBvp['ODOf'] = GBvp['ODOc']-GBvp['ODOa']
GBvp['CTf'] = GBvp['CTc']-GBvp['CTa']
GBvp = GBvp[['NRUTA','ODOa', 'ODOc','ODOf','CTa','CTc','CTf']]
GBvp= GBvp.drop_duplicates()

# Por último manipulamos las coordenadas geográficas, y los timestamp.

GBfin = M2bis[['NRUTA','LATa','LONGa','LATc','LONGc','TMPa','TMPc','MTR']] 
GBfin0 = GBfin.groupby(['NRUTA'])
GBfin['LATa'] =GBfin0['LATa'].transform("first")
GBfin['LONGa'] =GBfin0['LONGa'].transform("first")
GBfin['LATc'] =GBfin0['LATc'].transform("last")
GBfin['LONGc'] =GBfin0['LONGc'].transform("last")
GBfin['TMPa'] =GBfin0['TMPa'].transform("first")
GBfin['TMPc'] =GBfin0['TMPc'].transform("last")
GBfin= GBfin.drop_duplicates()



In [ ]:
# Finalmente unimos todos los datasets para generar el dataset final optimo para modelizar.

MAESTRO_RUTAS = pd.merge(GBfin,GBcond, on = 'NRUTA')
MAESTRO_RUTAS = pd.merge(MAESTRO_RUTAS,GBcs1, on ='NRUTA')
MAESTRO_RUTAS = pd.merge(MAESTRO_RUTAS,GBcs2, on = 'NRUTA')
MAESTRO_RUTAS = pd.merge(MAESTRO_RUTAS,GBcs3 , on = 'NRUTA')
MAESTRO_RUTAS = pd.merge(MAESTRO_RUTAS,GBvp, on = 'NRUTA')
MAESTRO_RUTAS = pd.merge(MAESTRO_RUTAS,GBrv, on = 'NRUTA')
MAESTRO_RUTAS = MAESTRO_RUTAS.drop_duplicates()

In [ ]:
MAESTRO_RUTAS['TMPa'] = pd.to_datetime(MAESTRO_RUTAS['TMPa'])
MAESTRO_RUTAS['TMPc'] = pd.to_datetime(MAESTRO_RUTAS['TMPc'])

In [ ]:
MAESTRO_RUTAS.head()

In [ ]:
MAESTRO_RUTAS.shape

In [ ]:
MAESTRO_RUTAS.columns

In [ ]:
MAESTRO_RUTAS.dtypes

Por último, añadimos una columna vital, y que nos indicará que tiempo se ha consumido en el desarrollo de la ruta (incluyendo los descansos dentro de la misma sin el camión en marcha y fuera del control de timestamp, ese valor será la columna TDsts).

In [ ]:
MAESTRO_RUTAS.dtypes

In [ ]:
MAESTRO_RUTAS['days']= MAESTRO_RUTAS['TMPc']-MAESTRO_RUTAS['TMPa']

# Transformamos el timedelta en horas.

MAESTRO_RUTAS['TIME_RUTA'] = MAESTRO_RUTAS['days'] / np.timedelta64(1, 'h')
del(MAESTRO_RUTAS['days'])
MAESTRO_RUTAS['CS3hrssum'] = MAESTRO_RUTAS['CS3rept']/60
MAESTRO_RUTAS['CS3hrsmean']=1/60
MAESTRO_RUTAS['TDsts']= MAESTRO_RUTAS['TIME_RUTA']- (MAESTRO_RUTAS['CS1hrssum']+MAESTRO_RUTAS['CS2hrssum']\
+MAESTRO_RUTAS['CS3hrssum'])


In [ ]:
MAESTRO_RUTAS.head(50)

In [ ]:
MAESTRO_RUTAS.shape

Finalmente los resultados se acumulan en unos archivos csv con objeto de ser utilzados (y evitar consumo computacional) si fuese preciso.

In [ ]:
MAESTRO_RUTAS.to_csv ('MAESTRO_RUTASTg.csv')

# Tarea (G)

In [ ]:
# Para todos el nexo de unión será la ruta

# Tipo de situaciones en cada ruta, y tiempo acumulado de las mismas. implicados en cada ruta GBmat
GBcs = M2Bis[['NRUTA','CS','TIME_HRS','COND']] 
GBcs0 = GBcs.groupby( [ 'NRUTA','CS','COND'] )
GBcs['CShrssum'] =GBcs0['TIME_HRS'].transform("sum")
GBcs['CShrsmean'] =GBcs0['TIME_HRS'].transform("mean")
GBcs['CSrept'] =GBcs0['CS'].transform("count")

# Generamos dataset con CS tipo 1
GBcs1 = GBcs[(GBcs['CS'] == 1)]
GBcs1 = GBcs1[['NRUTA','CS','CShrssum','CShrsmean','CSrept','COND']]
GBcs1.rename(columns={'CS': 'CS1','CShrssum':'CS1hrssum','CShrsmean':'CS1hrsmean','CSrept':'CS1rept'}, inplace=True)
GBcs1= GBcs1.drop_duplicates()

# Generamos dataset con CS tipo 2
GBcs2 = GBcs[(GBcs['CS'] == 2)]
GBcs2 = GBcs2[['NRUTA','CS','CShrssum','CShrsmean','CSrept','COND']]
GBcs2.rename(columns={'CS': 'CS2','CShrssum':'CS2hrssum','CShrsmean':'CS2hrsmean','CSrept':'CS2rept'}, inplace=True)         
GBcs2= GBcs2.drop_duplicates()

# Generamos dataset con CS tipo 3             
GBcs3 = GBcs[(GBcs['CS'] == 3)]
GBcs3 = GBcs3[['NRUTA','CS','CShrssum','CShrsmean','CSrept','COND']]
GBcs3.rename(columns={'CS': 'CS3','CShrssum':'CS3hrssum','CShrsmean':'CS3hrsmean','CSrept':'CS3rept'}, inplace=True)         
GBcs3= GBcs3.drop_duplicates()

# Acumulado variables restantes. Se realizara un promedio de cada una de ella.

GBrv = M2Bis[['NRUTA','COND','ALTa', 'ALTc', 'VELCa', 'HMa', 'HMc','CCa', 'CCc', 'TMa', 'RPMa','NFa', 'FRNa', 'FRNc',\
        'TRa', 'TRc', 'PMa', 'PMc','PDLa','PDLc', 'EMBa', 'EMBc']]

GBrv0 = GBrv.groupby( [ 'NRUTA','COND' ] )

# Asignamos promedios

cols = ['VELCa', 'TMa', 'RPMa','NFa','TRa']

for c in cols:
    GBrv[c] = GBrv0[c].transform("mean")
    
# Asignamos valores iniciales.

cols = ['ALTa', 'HMa','FRNa','PMa', 'PDLa', 'EMBa']

for c in cols:
    GBrv[c] = GBrv0[c].transform("first")

GBrv['CCa'] = GBrv0['CCa'].transform("sum")

# Asignamos valores finales.

cols = ['ALTc', 'HMc','FRNc','PMc', 'PDLc', 'EMBc']

for c in cols:
    GBrv[c] = GBrv0[c].transform("last")

GBrv['CCc'] = GBrv0['CCc'].transform("count") 

GBrv= GBrv.drop_duplicates()

GBrv['ALTf'] = GBrv['ALTc']-GBrv['ALTa']
GBrv['HMf'] = GBrv['HMc']-GBrv['HMa']
GBrv['CCf'] = (GBrv['CCa']/GBrv['CCc'])
GBrv['FRNf'] = GBrv['FRNc']-GBrv['FRNa'] # Este valor se entendería como porcentaje.
GBrv['PMf'] = GBrv['PMc']-GBrv['PMa']
GBrv['PDLf'] = GBrv['PDLc']-GBrv['PDLa']
GBrv['EMBf'] = GBrv['EMBc']-GBrv['EMBa']

GBrv = GBrv[['NRUTA','COND','ALTf', 'VELCa', 'HMf','CCf', 'TMa', 'RPMa','NFa', 'FRNf',\
        'TRa', 'PMf','PDLf', 'EMBf']]
GBrv= GBrv.drop_duplicates()

# Manipulamos las variables que mantuvimos como float. ODO (odómetro) y CT (consumo total), estableciendo valor inicial y valor final.

GBvp = M2Bis[['NRUTA','COND','ODOa', 'ODOc', 'CTa','CTc']]
GBvp0 = GBvp.groupby(['NRUTA','COND'])
GBvp['ODOa'] =GBvp0['ODOa'].transform("first")
GBvp['ODOc'] =GBvp0['ODOc'].transform("last")
GBvp['CTa'] =GBvp0['CTa'].transform("first")
GBvp['CTc'] =GBvp0['CTc'].transform("last")
GBvp= GBvp.drop_duplicates()
GBvp['ODOf'] = GBvp['ODOc']-GBvp['ODOa']
GBvp['CTf'] = GBvp['CTc']-GBvp['CTa']
GBvp = GBvp[['NRUTA','COND','ODOa', 'ODOc','ODOf','CTa','CTc','CTf']]

# Por último manipulamos las coordenadas geográficas, y los timestamp.

GBfin = M2Bis[['NRUTA','COND','LATa','LONGa','LATc','LONGc','TMPa','TMPc','MTR']] 
GBfin0 = GBfin.groupby(['NRUTA','COND'])
GBfin['LATa'] =GBfin0['LATa'].transform("first")
GBfin['LONGa'] =GBfin0['LONGa'].transform("first")
GBfin['LATc'] =GBfin0['LATc'].transform("last")
GBfin['LONGc'] =GBfin0['LONGc'].transform("last")
GBfin['TMPa'] =GBfin0['TMPa'].transform("first")
GBfin['TMPc'] =GBfin0['TMPc'].transform("last")
GBfin= GBfin.drop_duplicates()

In [ ]:
# Finalmente unimos todos los datasets para generar el dataset final optimo para modelizar.

MAESTRO_RUTASxcond = pd.merge(GBfin,GBcs1, on=['NRUTA', 'COND'])
MAESTRO_RUTASxcond = pd.merge(MAESTRO_RUTASxcond,GBcs2, on=['NRUTA', 'COND'])
MAESTRO_RUTASxcond = pd.merge(MAESTRO_RUTASxcond,GBcs3, on=['NRUTA', 'COND'])
MAESTRO_RUTASxcond = pd.merge(MAESTRO_RUTASxcond,GBvp, on=['NRUTA', 'COND'])
MAESTRO_RUTASxcond = pd.merge(MAESTRO_RUTASxcond,GBrv, on=['NRUTA', 'COND'])
MAESTRO_RUTASxcond = MAESTRO_RUTASxcond.drop_duplicates()

In [ ]:
MAESTRO_RUTASxcond['TMPa'] = pd.to_datetime(MAESTRO_RUTASxcond['TMPa'])
MAESTRO_RUTASxcond['TMPc'] = pd.to_datetime(MAESTRO_RUTASxcond['TMPc'])

In [ ]:
MAESTRO_RUTASxcond['days']= MAESTRO_RUTASxcond['TMPc']-MAESTRO_RUTASxcond['TMPa']
# Transformamos el timedelta en horas.
MAESTRO_RUTASxcond['TIME_RUTA'] = MAESTRO_RUTASxcond['days'] / np.timedelta64(1, 'h')
del(MAESTRO_RUTASxcond['days'])
MAESTRO_RUTASxcond['CS3hrssum'] = MAESTRO_RUTASxcond['CS3rept']/60
MAESTRO_RUTASxcond['CS3hrsmean']=1/60
MAESTRO_RUTASxcond['TDsts']= MAESTRO_RUTASxcond['TIME_RUTA']- (MAESTRO_RUTASxcond['CS1hrssum']\
+MAESTRO_RUTASxcond['CS2hrssum']+MAESTRO_RUTASxcond['CS3hrssum'])
MAESTRO_RUTASxcond = MAESTRO_RUTASxcond.drop_duplicates()

In [ ]:
MAESTRO_RUTASxcond.head()

In [ ]:
MAESTRO_RUTASxcond.shape

In [ ]:
MAESTRO_RUTASxcond.to_csv ('MAESTRO_RUTASTgxcond.csv')

# Proceso final. Exportación datasets para modelizar.

In [ ]:
M2.to_csv('DESARROLLO_RUTAST.csv') 
RUTAS.to_csv ('RUTASTg.csv')
MRUTAS.to_csv ('MRUTASTg.csv')

In [ ]:
print(datetime.datetime.now())